In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
kf = KFold(n_splits=3, shuffle=True)

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
combi = pd.concat([train.drop('author', axis=1), test], axis=0)

train = train.sample(frac=1.0) # shuffle

x = train.text
y = train.author

In [11]:
from sklearn.dummy import DummyClassifier

dum = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classify', DummyClassifier())])

cv = cross_val_score(dum, x, y, cv=4, scoring='accuracy')

print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

Mean score: 0.337402681485
Std Dev:    0.00756760360172


In [6]:
pl = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classify', MultinomialNB())
])

param_grid = [
    {
        'tfidf__max_df': np.arange(.01,.10,.01),
        'tfidf__min_df': [2,3,4],
        'tfidf__ngram_range': [(1,2)],
        'tfidf__norm': ['l2'],
        'classify__alpha': [.001, .01, .1],
    },
]

grid =\
GridSearchCV(pl, cv=3, n_jobs=-1, param_grid=param_grid, scoring='accuracy')\
.fit(x, y)

model = grid.best_estimator_
print(model)
cv = cross_val_score(model, x, y, cv=4, scoring='accuracy')

print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.089999999999999997, max_features=None,
        min_df=2, ngram_range=(1, 2), norm='l2', preprocessor=None,
 ...f=True, vocabulary=None)), ('classify', MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True))])
Mean score: 0.851422370809
Std Dev:    0.00198274978384
